<a href="https://colab.research.google.com/github/WRFitch/fyp/blob/main/src/fyp_demonstration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FYP Demonstration
### Will Fitch 1633241 
### Brunel University Department of Computer Science

This notebook is a demonstration of my final year project, in which I have used neural networks and transfer learning to infer highly detailed greenhouse gas information from satellite data. 

This notebook also needs serious rework to match the NEW WAY. 

## Setup 

#### Imports

In [ ]:
import ee
import folium
import os

from fastai.vision.all import *
from google.colab import drive
from IPython.display import Image
from osgeo import gdal
from PIL import Image

#### Parts that require human input 
_TODO rename to not suck_

In [ ]:
# Sometimes fastai doesn't want to play with colab, so we remove it and replace 
# it with a compliant version just to be sure. 
!pip uninstall -y fastai
!pip install -U --no-cache-dir fastai

In [ ]:
ee.Authenticate()
ee.Initialize()
drive.mount('/content/drive')

In [ ]:
# Import fyputil library
# EE stuff needs initialising first
%cd /content
!git clone https://github.com/WRFitch/fyp.git
%cd fyp/src/fyputil
import constants as c
import ee_constants as eec
import ee_utils as eeutil
import fyp_utils as fyputil
%cd /content

#### Parts that don't require human input 

In [ ]:
drive_path = c.drive_path
demo_dir = "demo_export"
demo_path = f"{drive_path}{demo_dir}"
model_name = "resnet152_MRGHG_Demo1"

# Add a dummy label script to fool fastai into letting us import the model. 
# We're not going to train the model further, so this is fine. 
# TODO update to use the other model with a decent dataframe, just in case. 
def getGhgsAsArr(img_path):
  return np.array()

model = load_learner(f"{c.model_dir}/{model_name}.pkl")

In [ ]:
# Parses postcode into coordinates (obviously only works in the UK)
def getCoordinatesFromPostcode(postcode):
  print(postcode)
  return (0,0)

def getGhgsFromCoords(coords):
  img_path = importImg(coords)
  print(img_path)
  ghgs = getGhgsFromImg(img_path)
  displayGhgs(ghgs)
  displayHealthDefects(ghgs)
  displayEnvEffects(ghgs)

# use image pipeline to download an image based on a point object defined by the 
# given coordinates
def importImg(coords):
  # get a square from the given image. 
  # TODO why is this buffer 500? the ideal pixel size should be 224, so 2240m. 
  polygon = ee.Geometry.Point(coords).buffer(500).bounds()

  name = f"{coords[0]}_{coords[1]}"
  tifname = f"{name}.tif"

  # check if image is cached before importing from GEE
  cachePath = f"{demo_path}/{name}.png"
  if os.path.isfile(cachePath): 
    print("image for these coordinates has already been imported!")
    return cachePath

  print(f"importing {tifname}")
  eeutil.exportGeotiff(s2_img, polygon, 10, demo_dir, name)
  tifpath = f"{demo_path}/{tifname}"
  while not os.path.isfile(tifpath):
    time.sleep(5)
  print(f"imported  {tifname}")
  
  # Convert GeoTIFF to PNG.
  fyputil.geotiffToPng(tifpath)
  fyputil.rmArtifact(f"{demo_path}/{name}.png.aux.xml")
  print(f"{tifpath} converted to PNG")

  importPath = f"{demo_path}/{name}.png"
  if os.path.isfile(importPath): return importPath

  print("image import failed")

def getGhgsFromImg(img_path):
  print("getting GHG concentrations from given image")
  display(Image.open(img_path))
  predictions = model.predict(img_path)[0]
  print(predictions)
  return predictions 

def displayGhgs(ghgs):
  print(ghgs)
  # Add ghgs to table
  # compare against average
  # display ghgs as a series of plots. 
  # for each ghg concentration, display health defects and mitigation strategies 
  # on table
  # print table

def displayHealthDefects(ghgs):
  print(ghgs)
  # for each ghg, compare against concentration. Depending on how high they are,
  # display the potential health effects and their likelihoods. 
  # return dict of ghg to string tuples of horrible effects and their likelihoods. 

def displayEnvEffects(ghgs):
  print(ghgs)
  # for each ghg, compare against concentration. Depending on how high they are, 
  # display the potential environmental effects and their likelihoods. 
  # return dict of ghg to string tuples of horrible effects and their likelihoods. 

# Run all the above code to ensure it works fine and is initialised ok. A lot of 
# python efficiency requires precaching this stuff into memory, so on the day 
# run this beforehand
def demoTest():
  getCoordinatesFromPostcode("UB8 3PH")  

demoTest()

UB8 3PH


## Demonstration

### Visualisation 

In [ ]:
eec.map 

### Model Interpolation

In [ ]:
#@title Insert Postcode

# list of coords 
brunel = (-0.47278354461716354, 51.53325658151181)

getGhgsFromCoords(brunel)